<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [15]:
## Task 1: Mark all launch sites on a map

#spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv")
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)

site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=5)

for index, row in launch_sites_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [6]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [17]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='blue',
        fill=True,
        fill_color='blue',
        popup=row['Launch Site']
    ).add_to(site_map)


site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [23]:
# Task 2: Mark the success/failed launches for each site on the map

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
spacex_df['Outcome'] = spacex_df['class'].apply(lambda x: 'Success' if x == 1 else 'Failure')
#spacex_df['Outcome'] = spacex_df['Class'].apply(lambda x: 'Success' if x == 1 else 'Failure')

def outcome_color(Outcome):
    if 'Success' in Outcome:
        return 'green'
    else:
        return 'red'


for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=outcome_color(row['Outcome'])),
        popup=row['Launch Site'] + " / " + row['Outcome']
    ).add_to(site_map)

site_map


In [21]:
print(spacex_df.columns)



Index(['Launch Site', 'Lat', 'Long', 'class'], dtype='object')


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [24]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Outcome
46,KSC LC-39A,28.573255,-80.646895,1,Success
47,KSC LC-39A,28.573255,-80.646895,1,Success
48,KSC LC-39A,28.573255,-80.646895,1,Success
49,CCAFS SLC-40,28.563197,-80.576820,1,Success
50,CCAFS SLC-40,28.563197,-80.576820,1,Success
51,CCAFS SLC-40,28.563197,-80.576820,0,Failure
52,CCAFS SLC-40,28.563197,-80.576820,0,Failure
53,CCAFS SLC-40,28.563197,-80.576820,0,Failure
54,CCAFS SLC-40,28.563197,-80.576820,1,Success
55,CCAFS SLC-40,28.563197,-80.576820,0,Failure


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [27]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [29]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [33]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [36]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    # Haversine formula
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    distance = R * c
    return distance
# Launch site: LC-40 (28.5623° N, -80.5774° W)
# Closest city: Cocoa Beach (28.3200° N, -80.6076° W)

launch_lat = 28.5623
launch_lon = -80.5774
city_lat = 28.3200
city_lon = -80.6076

distance_km = haversine_distance(launch_lat, launch_lon, city_lat, city_lon)
print(f"Distance to Cocoa Beach: {distance_km:.2f} km")



Distance to Cocoa Beach: 27.10 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [37]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [38]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [39]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat = 28.562302
launch_site_lon = -80.577356 

coastline_lat = 28.56367
coastline_lon = -80.57163

import math

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    return R * c
    
distance_coastline = calculate_distance(28.562302, -80.577356, 28.56367, -80.57163)
print(f"Distance to coastline: {distance_coastline:.2f} km")

Distance to coastline: 0.58 km


In [40]:
import folium
from folium import Marker
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Coordinates for Cape Canaveral LC-40 (Launch site)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates for closest coastline
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered at the launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add a marker at the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site: LC-40',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Add a marker at the coastline point
folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Add a line between the two points
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='orange'
).add_to(site_map)

# Add a distance marker with DivIcon
distance_marker = folium.Marker(
    [ (launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2 ],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
    )
)
site_map.add_child(distance_marker)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [42]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, 

import folium

# Coordinates for the launch site and coastline point
launch_site_coords = [28.5618571, -80.577366]  # Example: CCAFS Launch Complex 40
coastline_coords = [28.573255, -80.646895]     # Example coastline coordinate near Florida

# Calculate the midpoint for centering the map
map_center = [
    (launch_site_coords[0] + coastline_coords[0]) / 2,
    (launch_site_coords[1] + coastline_coords[1]) / 2
]

# Create the Folium map centered at the midpoint
site_map = folium.Map(location=map_center, zoom_start=10)

# Add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup='Launch Site',
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(site_map)

# Add a marker for the coastline point
folium.Marker(
    location=coastline_coords,
    popup='Coastline Point',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Draw a green polyline between the launch site and coastline point
line = folium.PolyLine(
    locations=[launch_site_coords, coastline_coords],
    weight=3,
    color='green',
    opacity=0.8
)
line.add_to(site_map)

# Display the map (if using Jupyter Notebook)
site_map

# site_map.save('launch_site_coastline_map.html')
# print("Map saved as 'launch_site_coastline_map.html'. Open this file in a browser to view the map.")

site_map.add_child(line)

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


Your updated map with distance line should look like the following screenshot:


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [45]:
from folium import Marker
from folium.plugins import MousePosition
from folium.features import DivIcon
import math

# Function to calculate haversine distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    radius = 6371  # km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return radius * c

# Coordinates
launch_site_coord = [28.562302, -80.577356]

# Replace the following with your coordinates from MousePosition
city_coord = [28.61000, -80.80700]
rail_coord = [28.56350, -80.57000]
highway_coord = [28.58000, -80.65000]

# Initialize the map again if needed
site_map = folium.Map(location=launch_site_coord, zoom_start=10)

# Add launch site marker
folium.Marker(launch_site_coord, popup="Launch Site").add_to(site_map)

# --- Function to draw a line and show distance ---
def draw_proximity_line(map_obj, point_coord, label):
    dist = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                              point_coord[0], point_coord[1])

    # Add a marker for the proximity point
    folium.Marker(point_coord, popup=label).add_to(map_obj)

    # Add a distance label
    folium.Marker(
        point_coord,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: red;"><b>{label}: {dist:.2f} km</b></div>',
        )
    ).add_to(map_obj)

    # Draw the line
    line = folium.PolyLine(locations=[launch_site_coord, point_coord], color='green', weight=2)
    map_obj.add_child(line)

# Draw for each point
draw_proximity_line(site_map, city_coord, "Closest City")
draw_proximity_line(site_map, rail_coord, "Closest Rail")
draw_proximity_line(site_map, highway_coord, "Closest Highway")

# Show the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
